In [165]:
from astropy.time import Time
import datetime as dt
import astropy.units as u
import warnings
from astropy.utils.exceptions import AstropyUserWarning, AstropyWarning
from astropy.table import vstack, Table
import numpy as np
from transit_prediction.utils.list_of_constraints import List_of_constraints 

# Import astroplan tools and constraints
# Other constraints are available
from astroplan import (FixedTarget, Observer, EclipsingSystem,
                       PrimaryEclipseConstraint, is_event_observable,
                       AtNightConstraint, AltitudeConstraint,
                       TimeConstraint, AirmassConstraint, PhaseConstraint)

from masterfile import MasterFile

In [241]:
def min_start_time(constraints_list, obs, target, time, dt=1*u.min):
    
    time = Time(time)
    evening = obs.sun_set_time(time, which='previous')

    time_grid = np.arange(evening.jd, time.jd, dt.to('day').value) * u.d
    time_grid = Time(time_grid, format='jd')
    
    index = is_event_observable(constraints_list,
                                obs, target,
                                times=time_grid).squeeze()
    
    return time_grid[index][0]

def max_end_time(constraints_list, obs, target, time, dt=1*u.min):
    
    time = Time(time)
    morning = obs.sun_rise_time(time, which='next')

    time_grid = np.arange(time.jd, morning.jd, dt.to('day').value) * u.d
    time_grid = Time(time_grid, format='jd')
    
    index = is_event_observable(constraints_list,
                                obs, target,
                                times=time_grid).squeeze()
    
    return time_grid[index][-1]

def min_start_times(constraints_list, obs, target, times, dt=1*u.min):
    
    out = []
    for time in times:
        out.append(min_start_time(constraints_list, obs, target, time, dt=1*u.min))
        
    return Time(out)

def max_end_times(constraints_list, obs, target, times, dt=1*u.min):
    
    out = []
    for time in times:
        out.append(max_end_time(constraints_list, obs, target, time, dt=1*u.min))
        
    return Time(out)

# Get planets info

In [242]:
target_list = ['51 Peg', 'ups And b']  # Stefan

In [243]:
data = MasterFile.read().by_plName(*target_list)

# Inputs

In [294]:
t1, t2 = Time(['2019-08-01 00:00', '2020-01-31 00:00'])
site = 'cfht'
# abs_baseline = 30 * u.min
# baseline = None #2
# See astroplan available constraints
constraints = [AtNightConstraint.twilight_nautical(),
               AirmassConstraint(max=2.),
               TimeConstraint(t1,t2)
              ]
constraints_list = List_of_constraints(constraints)
supp_cols = ['st_j','st_h','ra','dec','pl_eqt','st_teff']
n_eclipses = 5000

## Special inputs for phase

In [311]:
# phase_range = [0.35, 0.49]  # One range at the time
phase_range = [0.51, 0.65]
obs_time = 5 * u.h
dt_grid = 0.5*u.h

In [296]:
# constraints_list = [*constraints, PhaseConstraint()]

# Add info about planets

In [297]:
from transit_prediction.utils.table_edited import Column 

In [298]:
phase_zero = [2458708.83240,2458710.05353] * u.d

In [270]:
data.add_column(Column(phase_zero, name='pl_phase_curve_zero', 
       description='equivalent to mid-transit time for a non eclipsing system. Use with caution')
               )

ValueError: Duplicate column names

In [312]:
required_info = data['pl_name','pl_orbper','pl_phase_curve_zero']

In [313]:
required_info.show_in_notebook()

idx,pl_name,pl_orbper,pl_phase_curve_zero
,,d,d
0,51 Peg b,4.230785,2458708.8324
1,ups And b,4.617033,2458710.05353


In [314]:
# Define needed quantities based on planets infos
# Must be quatities arrays (astropy)
# Here we use a given astropy Table (data) to get the infos

epoch, period = data.cols_2_qarr('pl_phase_curve_zero', 'pl_orbper')
epoch = Time(epoch, format='jd')
pl_name = data['pl_name']


observing_time = t1
obs = Observer.at_site(site)

In [315]:
col_names = ('pl_name',
             'Obs_start',
             'Phase_start',
             'Obs_end',
             'Phase_end',
             'mid_tr',
             'AM_mid_tr',
             'moon',
             *supp_cols
            )
meta = {'Time_limits': [t1, t2],
        'Target_list': pl_name.tolist(),
        'Site': site,
        'Phase_range': phase_range,
        **constraints_list.show()
       }

In [316]:
from astroplan import PeriodicEvent

In [317]:
full_table = Table()
for itar in range(len(pl_name)):

    # -------------------------
    # Steps to predict transits
    # -------------------------
    try:
        target = FixedTarget.from_name(pl_name[itar])
    except NameResolveError as e:
        print(e)
        try_name = ' '.join(pl_name[itar].split(' ')[:-2])
        print("Trying with {}".format(try_name))
        try:
            target = FixedTarget.from_name(try_name)
        except NameResolveError as e:
            print(e)
            print('Searching in masterfile for RA and dec')
            try_radec = MasterFile.read().by_plName(pl_name[itar])
            ra = try_radec['ra'].quantity
            dec = try_radec['dec'].quantity
            coord = SkyCoord(ra=ra, dec=dec)
            target = FixedTarget(coord=coord,
                                 name=pl_name[itar])
            

    
    dt_grid = 0.5*u.h
    d_phase = (dt_grid /period[itar]).decompose().value
    [p1, p2] = phase_range
    p1 += d_phase/10  # Make sure it's not on the boundary
    p2 -= d_phase/10
    phase_grid = np.arange(p1, p2, d_phase)
    phase_grid = phase_grid*period[itar] + epoch[itar]
    
    t_grid = []
    for phase in phase_grid:
        sys = EclipsingSystem(primary_eclipse_time=phase,
                              orbital_period=period[itar])
        t_temp = sys.next_primary_eclipse_time(observing_time,
                                               n_eclipses=n_eclipses)
        t_grid.append(t_temp.jd)
    t_grid = Time(t_grid, format='jd').T

    
    if t_grid[-1,-1] < t2:
        warnings.warn('end time ('+t2.value +
                      ') is passed the last computed event time (' +
                      t_mid[-1].value+')\n' +
                     '\t You can change the n_eclipse kwarg ' +
                     'value or choose a different window (start or end time)',
                     AstropyUserWarning
                    )

    t_grid = t_grid[(t_grid < t2).any(axis=1)]
    
    events = []
    for grid in t_grid:
        index = is_event_observable(constraints_list, obs, target, times=grid).squeeze()
        if index.any():
            events.append(np.mean(grid[index].jd))
    events = Time(events, format='jd')
    
    # Finally add phase constraint
    sys = PeriodicEvent(epoch=epoch[itar], period=period[itar])
    
    final_constraints = [*constraints_list,
        PhaseConstraint(sys, *phase_range)]
    
    obs_start = min_start_times(final_constraints, obs, target, events)
    obs_end = max_end_times(final_constraints, obs, target, events)
    baseline = obs_end - obs_start
    t_mid = obs_start + baseline/2
    
    index = (obs_end - obs_start) > obs_time

    # -------------------
    # End of steps to predict transits
    # -------------------

    # Put the infos in a table and stack it to the full table
    if index.any():
        name = np.repeat(target.name,index.sum()).astype(str)
        moon = obs.moon_illumination(t_mid[index])
        phase_start = sys.phase(obs_start[index])
        phase_end = sys.phase(obs_end[index])
        AM_mid = obs.altaz(t_mid[index], target).secz
        supp = [np.repeat(data[key][itar],index.sum())
                for key in supp_cols]
        cols = [name,
                obs_start[index].iso,
                phase_start,
                obs_end[index].iso,
                phase_end,
                t_mid[index].iso,
                AM_mid,
                moon,
                *supp
               ]
        table_sys = Table(cols, names=col_names, masked=True)
        full_table = vstack([table_sys, full_table])
    else:
        warnings.warn('No event found for '+sys.name, AstropyUserWarning)

if full_table:
    full_table.sort('mid_tr')
    full_table.meta = meta
else:
    warnings.warn('No event found at all', AstropyUserWarning)

In [318]:
full_table[full_table['moon']>0.5]

pl_name,Obs_start,Phase_start,Obs_end,Phase_end,mid_tr,AM_mid_tr,moon,st_j,st_h,ra,dec,pl_eqt,st_teff
,,,,,,,,,,,,,
str9,str23,float64,str23,float64,str23,float64,float64,float64,float64,float32,float32,float64,float64
51 Peg b,2019-08-11 07:44:54.782,0.5250187084839435,2019-08-11 15:10:24.845,0.5981436086641285,2019-08-11 11:27:39.814,1.0099745124866908,0.8535600541530874,4.655,4.234,344.36658,20.76883,--,5793.0
ups And b,2019-08-12 10:10:17.578,0.538726398462137,2019-08-12 15:10:18.282,0.5838509376958902,2019-08-12 12:40:17.930,1.1908999451470768,0.9189839389084999,3.175,2.957,24.19934,41.40546,--,--
ups And b,2019-09-09 08:20:16.416,0.5866791991406927,2019-09-09 15:19:28.189,0.6497301859342143,2019-09-09 11:49:52.303,1.1018251209211758,0.8132332986630775,3.175,2.957,24.19934,41.40546,--,--
51 Peg b,2019-09-10 05:47:22.565,0.596608758290761,2019-09-10 11:11:43.614,0.6498479620710113,2019-09-10 08:29:33.089,1.0795853521200622,0.8730751361035589,4.655,4.234,344.36658,20.76883,--,5793.0
51 Peg b,2019-09-14 05:31:45.062,0.5394950733588774,2019-09-14 14:05:03.437,0.6237495735550396,2019-09-14 09:48:24.250,1.0001689112360876,0.9979119147030264,4.655,4.234,344.36658,20.76883,--,5793.0
51 Peg b,2019-09-18 08:05:05.613,0.5101158570955222,2019-09-18 13:48:23.262,0.5664644342324764,2019-09-18 10:56:44.437,1.0632940875556853,0.8395982446429641,4.655,4.234,344.36658,20.76883,--,5793.0
ups And b,2019-09-18 07:44:05.616,0.530541198187666,2019-09-18 15:21:31.492,0.5993430890150949,2019-09-18 11:32:48.554,1.0878054939434774,0.83780031131155,3.175,2.957,24.19934,41.40546,--,--
ups And b,2019-10-11 07:30:46.391,0.5100918563624633,2019-10-11 15:08:09.748,0.578887431810984,2019-10-11 11:19:28.069,1.0878024751141289,0.946234296491285,3.175,2.957,24.19934,41.40546,--,--


In [319]:
# Save to CFHT form
table_cfht = full_table["pl_name","ra","dec","Obs_start","Obs_end"][full_table['moon']>0.5]
table_cfht.rename_column('Obs_start', 'Date start (UT)')
table_cfht.rename_column('Obs_end', 'Date end (UT)')

In [320]:
table_cfht

pl_name,ra,dec,Date start (UT),Date end (UT)
str9,float32,float32,str23,str23
51 Peg b,344.36658,20.76883,2019-08-11 07:44:54.782,2019-08-11 15:10:24.845
ups And b,24.19934,41.40546,2019-08-12 10:10:17.578,2019-08-12 15:10:18.282
ups And b,24.19934,41.40546,2019-09-09 08:20:16.416,2019-09-09 15:19:28.189
51 Peg b,344.36658,20.76883,2019-09-10 05:47:22.565,2019-09-10 11:11:43.614
51 Peg b,344.36658,20.76883,2019-09-14 05:31:45.062,2019-09-14 14:05:03.437
51 Peg b,344.36658,20.76883,2019-09-18 08:05:05.613,2019-09-18 13:48:23.262
ups And b,24.19934,41.40546,2019-09-18 07:44:05.616,2019-09-18 15:21:31.492
ups And b,24.19934,41.40546,2019-10-11 07:30:46.391,2019-10-11 15:08:09.748
ups And b,24.19934,41.40546,2019-10-16 05:54:49.304,2019-10-16 13:48:41.399


In [323]:
file_name = '/home/adb/Doctorat/transit_predictions/CFHT_window_{}_{}.csv'

In [324]:
full_table.write(file_name.format(*meta['Phase_range']),
                 format='ascii', delimiter=',')

In [289]:
file_name = '/home/adb/Doctorat/transit_predictions/list_of_phase_stefan_{}_{}.ecsv'

In [292]:
full_table.write(file_name.format(*meta['Phase_range']),
                 delimiter=',')

In [291]:
full_table.meta

{'Time_limits': [<Time object: scale='utc' format='iso' value=2019-08-01 00:00:00.000>,
  <Time object: scale='utc' format='iso' value=2020-01-31 00:00:00.000>],
 'Target_list': ['51 Peg b', 'ups And b'],
 'Site': 'cfht',
 'Phase_range': [0.51, 0.65],
 'AtNightConstraint': {'max_solar_altitude': <Quantity -12. deg>,
  'force_pressure_zero': True},
 'AirmassConstraint': {'min': 1, 'max': 2.0, 'boolean_constraint': True},
 'TimeConstraint': {'min': <Time object: scale='utc' format='iso' value=2019-08-01 00:00:00.000>,
  'max': <Time object: scale='utc' format='iso' value=2020-01-31 00:00:00.000>}}

In [325]:
!ls /home/adb/Doctorat/transit_predictions/

CFHT_window_0.35_0.49.csv	 list_of_eclipses_KELT7.ecsv
CFHT_window_0.51_0.65.csv	 list_of_eclipses_mascara-1-2.ecsv
CFHT_window_3targets_after.csv	 list_of_eclipses_WASP33.ecsv
CFHT_window_3targets_before.csv  list_of_phase_stefan_0.35_0.49.ecsv
CFHT_window_KELT7.csv		 list_of_phase_stefan_0.4_0.48.ecsv
CFHT_window_KELT7.ecsv		 list_of_phase_stefan_0.51_0.65.ecsv
CFHT_window_WASP33_after.csv	 list_of_phase_stefan_0.52_0.6.ecsv
CFHT_window_WASP33_before.csv	 list_of_transits_Boucher.ecsv
list_of_eclipses_GeertJan.ecsv	 list_of_transits_mascara-1-2.ecsv
